Script for database project 3. 

In [1]:
import pymysql 
import getpass
from IPython.display import clear_output
import datetime

In [2]:
class STUDENT:
    def __init__(self,ID,Name,Password,Address):
        self.ID = ID
        self.Name = Name
        self.Password = Password
        self.Address = Address
        
    def info(self):
        return (self.ID,self.Name,self.Password,self.Address)

In [3]:
def CheckDatabaseOpen(dbname):
    mydb = pymysql.connect("localhost","root","password")
    mycursor = mydb.cursor()
    mycursor.execute("SHOW DATABASES")
    for name in mycursor:
#         print(name)
        if name[0] == dbname:
            mydb.close()
            print(">DATABASE ",dbname, "is open!!!")
            return
    print(">DATABASE ",dbname, "is not open!!!")
    mydb.close()
    return

dbname = 'project3-nudb'
CheckDatabaseOpen(dbname)

>DATABASE  project3-nudb is open!!!


In [4]:
def Login():
    while(1):
        print("please enter your user name:")
        usr = input()
        pwd = getpass.getpass(prompt='Password: ')
        if mycursor.execute('select * from student where Name = %s and Password = %s',(usr,pwd)) > 0:
            stu_info = mycursor.fetchone()
            my_stu = STUDENT(*stu_info)
            clear_output()
            print('welcome to your student menu, %s' % usr)
            return my_stu
        else:
            clear_output()
            print("user name not exist or passowrd not correct!")


In [ ]:
def LoadStudentMenu()

In [5]:
mydb = pymysql.connect("localhost","root","password","project3-nudb")
mycursor = mydb.cursor()

In [ ]:
my_stu =Login()

user name not exist or passowrd not correct!
please enter your user name:


In [ ]:
now = datetime.datetime.now()
print(now.year,now.month)

In [ ]:
mycursor.execute('select * from student, transcript where student.id = transcript.Studid and Semester = "Q1" AND Year="2018" and Grade is NULL')
for x in mycursor:
    print(x)

In [ ]:
len(x)
print(x)

In [ ]:
print(user_name)

In [ ]:
import getpass
user = getpass.getuser()
username = getpass.getpass('Username: %s' % user)
pwd = getpass.getpass(prompt='Password: ')